In [1]:
מציאת סימון שבילים וכיוון תזוזה ביחס אליו

SyntaxError: invalid syntax (957182362.py, line 1)

In [ ]:
import cv2
import numpy as np

#פונקציה לסיבוב תמונות
def rotateImage(sub_image):
    height, width = sub_image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), 45, 1)
    sub_image_top_right = cv2.warpAffine(sub_image, rotation_matrix, (width, height))

    rotation_matrix1 = cv2.getRotationMatrix2D((width / 2, height / 2), -45, 1)
    sub_image_top_left = cv2.warpAffine(sub_image, rotation_matrix1, (width, height))

   
    sub_image_horizontal = cv2.rotate(sub_image, cv2.ROTATE_90_CLOCKWISE)

    sub_image_vertical=sub_image
    
    return sub_image_top_right, sub_image_top_left,sub_image_horizontal,sub_image_vertical
    



def find_object_direction(video_path, sub_image_path):

    sub_image = cv2.imread(sub_image_path)
#imread הופך תמונה למערך

    video = cv2.VideoCapture(video_path)
#VideoCapture ניתוח וידאו וחלוקה לפריימים
    
    
#קורא כל פעם פריים אחד מהוידאו לצורך חישוב
    ret, frame_prev = video.read()
#ret מחזיר אמת או שקר אם הפריים נקרא בהצלחה
#frame_prev מחזיר מערך המייצג את ערכי הפקסלים של הפריים שנקרא     
 #אתחול הערך למעלה שמאלה
    top_left_prev = None
    array_x = []
    array_y = []
    #מערך מונים לצורך כיוון התזוזה
    top_right1=0
    bottom_right1=0
    bottom_left1=0
    top_left1=0
    bottom1=0
    top1=0
    right1=0
    left1=0    
#אתחול כל משתני הכיוונים
    dir_x_total = 0
    dir_y_total = 0
    frame_count = 0    
#כל עוד שיש ברט ערך אמת שהמסגרת נקראה תבצע
    while ret:
        ret, frame_curr = video.read()
#אם אין עוד פריימים תצא
        if not ret:
            break
        
#זימון הפונקציה שמסובבת תמונות
        sub_image_top_right, sub_image_top_left,sub_image_horizontal,sub_image_vertical=rotateImage(sub_image)
        #מציאת התת תמונה לפי כל הכיוונים
        result_horizontal = cv2.matchTemplate(frame_curr, sub_image_horizontal, cv2.TM_CCOEFF_NORMED)     
        a1,max_horizontal,c1, max_loc_horizontal = cv2.minMaxLoc(result_horizontal)

        result_vertical = cv2.matchTemplate(frame_curr, sub_image_vertical, cv2.TM_CCOEFF_NORMED)     
        a2, max_vertical,c2, max_loc_vertical = cv2.minMaxLoc(result_vertical)

        result_top_right = cv2.matchTemplate(frame_curr, sub_image_top_right, cv2.TM_CCOEFF_NORMED)     
        a3, max_top_right,c3, max_loc_top_right = cv2.minMaxLoc(result_top_right)

        result_top_left = cv2.matchTemplate(frame_curr, sub_image_top_left, cv2.TM_CCOEFF_NORMED)     
        a4, max_top_left,c4, max_loc_top_left = cv2.minMaxLoc(result_top_left)

        #היכן מצא את ההתאמה הטובה ביותר
        print(max_horizontal,max_vertical,max_top_right,max_top_left)
        
        max_value=max(max_horizontal,max_vertical,max_top_right,max_top_left)
        print(max_value)
        if max_horizontal==max_value:
            top_left=max_loc_horizontal
        elif max_vertical==max_value:
            top_left=max_loc_vertical
        elif max_top_right==max_value:
            top_left=max_loc_top_right
        else:
            top_left=max_loc_top_left

 # top_left פינה למעלה שמאלה- מכילה את הערך המקסימלי      
      
#bottom_right פינה למטה ימינה
        bottom_right = (top_left[0] + sub_image.shape[1], top_left[1] + sub_image.shape[0])
#התמונה המינימלית
        cropped_image = frame_curr[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
#חיתוך התמונה
        cv2.imshow('Result', cropped_image)

        if top_left_prev is not None:
            direction_x = top_left[0] - top_left_prev[0]
            array_x.append(direction_x)
#חישוב הקצה העליון הנוכחי לעומת הקצה של הפריים הקודם
# כך מזהה את כיוון התנועה
            direction_y = top_left[1] - top_left_prev[1]
            array_y.append(direction_y)
            dir_x_total += direction_x
            dir_y_total += direction_y
            frame_count += 1
        
# מעדכן את הפריים הקודם- top_left_prev
        top_left_prev = top_left

#לצורך יציאה מלולאה כשלוחצים על התו הזה יוצא מהלולאה
        if cv2.waitKey(1) == ord('q'):
            break
#שחרור כל החלונות תצוגה שנפתחו
    video.release()
    cv2.destroyAllWindows()   
    for i in range(len(array_x)):
       # print( array_x[i] , array_y[i])

        if array_x[i]>0 and array_y[i]>0:
           bottom_right1=bottom_right1+1
        elif array_x[i] > 0 and array_y[i] < 0:
             top_right1= top_right1+1
        elif array_x[i] < 0 and array_y[i] > 0:
            bottom_left1=bottom_left1 +1
        elif array_x[i] < 0 and array_y[i] < 0:
             top_left1=top_left1+1
        elif array_x[i] > 0:
            right1=right1+1
        elif array_x[i] < 0:
            left1=left1+1
        elif array_y[i] > 0:
            bottom1=bottom1+1
        elif array_y[i] < 0:
            top1=top1+1
        else:
            direction = "stationary"
   #מציאת הערך המקסימלי והדפסתו
    directions = [bottom_right1,top_right1, bottom_left1, top_left1,right1,left1,bottom1, top1]
    directionsName = ['bottom_right1','top_right1', 'bottom_left1', 'top_left1','right1','left1','bottom1', 'top1']
    maxValue=-1000
    maxName=""
    for index, dir in enumerate(directions):
        if dir > maxValue:
            maxValue = dir
            maxName = directionsName[index]                
    print(f"Max direction: {maxName} with value {maxValue}") 
    
    
find_object_direction("./right.mp4","./tat.png")


In [ ]:
מציאת הכיוון אותו מראה הסימון שבילים

In [ ]:

#מדפיס את כל המטריצה
# סריקה אחת לכל הכיוונים
#מדפיס לפי שורה
import cv2
import numpy as np

def segment_image(image, num_segments):
    # עצב מחדש את התמונה למערך דו-ממדי של פיקסלים
    pixels = image.reshape(-1, 3).astype(np.float32)

    # בצע אשכולות K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, num_segments, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # המר את המרכזים למספרים שלמים
    centers = np.uint8(centers)

    # עצב מחדש את התוויות כך שיתאימו לצורת התמונה המקורית
    segmented_image = centers[labels.flatten()].reshape(image.shape)

    # צור מטריצה ​​של סיווג קבוצתי עבור כל פיקסל
    group_matrix = labels.reshape(image.shape[:2])

    return segmented_image, group_matrix

#מוצאת רצפים
def sequences(group_matrix):
    all_sequences = []  # Initialize a list to store sequences for all rows
    
    # Iterate through each row in the matrix
    for row_index, row in enumerate(group_matrix):
        current = None
        start = None
        length = 0
        sequences = []

        # Repeat each value in the row
        for i, value in enumerate(row):
            if value != current:
                if current is not None:
                    sequences.append((row_index, start, length, current))
                current = value
                start = i
                length = 1
            else:
                length += 1
        
        sequences.append((row_index, start, length, current))
        
        all_sequences.append(sequences)  # Append sequences for this row to the list
    
    # Sort sequences by length and return only the three longest sequences
    all_sequences_sorted = []
    for row_sequences in all_sequences:
        row_sequences_sorted = sorted(row_sequences, key=lambda x: -x[2])[:3]  # Sort by length and get the 3 longest sequences
        # Sort the 3 longest sequences by start index within the row
        row_sequences_sorted = sorted(row_sequences_sorted, key=lambda x: x[1])
        all_sequences_sorted.append(row_sequences_sorted)
    
    # Print values for each line
    # for row_index, row_sequences in enumerate(all_sequences_sorted):
    #     print(f"Sequences for Row {row_index}:")
    #     for sequence in row_sequences:
    #         row_index, start, length, value = sequence
    #         matrix_position = (row_index, start)
    #         print(f"Matrix Position: {matrix_position}, Length: {length}, Value: {value}")
    
    return all_sequences_sorted




#מוצאת את הרצף הארוך ביותר
def long_sequence(specific_line_sequences):  
    #הסבר המשתנים-
    #longest_sequence - tuple שמכיל מידע על הרצף הארוך ביותר שנמצא בשורה שצוינה. מבנה הטפול הוא (row_index, start, length, value),
    #longest_sequence_length - מכיל את אורך הרצף הארוך ביותר שנמצא בשורה שצוינה
    longest_sequence = None
    longest_sequence_length = 0
    long = []
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        long.append(length)
        #הרצף הארוך ביותר
    longest_sequence_length = max(long)  
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        if length == longest_sequence_length:
            longest_sequence=sequence

    # print(longest_sequence)
    # print(longest_sequence_length)
    return longest_sequence


#מוצאת את הרצף במקום אינדקס
def sequence_in_index(specific_line_sequences, index): 
    sequences_count = 0
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        if length > 15:
            sequences_count += 1
            if sequences_count == index:
                return start, length, value
 
    return 100000, 0, 5

# #מוצא 3 רצפים בערך באותו גודל ומחזיר מיקום התחלה וערך
# def find_similar_sequences(specific_line_sequences):
#     sequences_count = 0
#     start_positions = []
#     values = []
#     for sequence in specific_line_sequences:
#         row_index, start, length, value = sequence
#         if 10 <= length <= 60:  
#             sequences_count += 1
#             start_positions.append(start)
#             values.append(value)
            
#             if sequences_count == 3:
#                 return start_positions, values
    
#     return None, None

#פונקצית למציאת שיפוע- חיובי שלילי או שווה
def Incline (third_line, index_of_sequence):
    current_difference = 0
    prev_difference = 0
    positive = 0
    negative = 0
    equal = 0
    
    #רץ מהשליש השני של התמונה עד סוף השליש השני בקפיצות של 5
    for index in range(third_line, (third_line*2)):
         current_difference = (sequence_in_index (all_sequences[index], index_of_sequence))[0]
         result= current_difference - prev_difference
         if result > 0:
             positive += 1
         elif result < 0:
             negative += 1
         else:
             equal += 1
         prev_difference = current_difference 
    return positive, negative, equal 

def compare(num1, num2):
    if abs(num1 - num2) <= 7:
        return True
    else:
        return False

#מוצאת כיוון      
def direction(all_sequences, num_col, line_index, third_line,seq_index) :
    print("from function")
    positive = 0
    negative = 0
    equal = 0
 #עבור רוחב
    col= 0.8 * num_col
    if (long_sequence(all_sequences[line_index])[2]>=col and long_sequence(all_sequences[line_index+20])[2]>=col or
        long_sequence(all_sequences[line_index+20])[2]>=col and long_sequence(all_sequences[line_index-20])[2]>=col):
        # אם הרצף הארוך ביותר של השורה האמצעית גדול מ80% מהאורך של השורה וגם הרצף ב50 שורות קודם ו50 שורות אחרי 
        return("רוחב")
 #עבור אלכסונים 
    positive, negative, equal = Incline (third_line, seq_index)
    print (equal)
    print (positive)
    print (negative)
    if equal >= positive and equal>= negative:
         positive, negative, equal = Incline (third_line, seq_index+1)
         print (equal)
         print (positive)
         print (negative)
         if equal >=positive and  equal>= negative:
             return ("אורך")
    
    elif  positive >= equal and positive>= negative:
        positive, negative, equal = Incline (third_line, seq_index)
        if  positive >= equal and positive>= negative:
            return ("אלכסון לשמאל")
    elif  negative >= equal and negative >= positive:
          positive, negative, equal = Incline (third_line, seq_index)
          if negative >= equal and negative >= positive:
              return ("אלכסון לימין")
    return "not found"


image = cv2.imread('./pppp.jpg')

num_segments =3
segmented_image, group_matrix = segment_image(image, num_segments)
#segmented_image-מטריצה שמייצגת את התמונה המסווגת
#group_matrix- מטריצה שמייצגת עבור כל פיקסל בתמונה לאיזו קבוצה הוא משתייך

np.set_printoptions(threshold=np.inf)

# # הדפס כל שורה בשורה אחת ארוכה
# for row in group_matrix:
#     print(*row)
#     print("next")
# print ()



 #חישוב כמה העמודות
num_col = len(group_matrix[0])

#זימון פונקצית הרצפים
all_sequences = sequences(group_matrix)
#זימון פונקצית הרצף הארוך
line_index = round(len(group_matrix)/2)
#חישוב מיקום השליש השני במטריצה
third_line = len(group_matrix)//3
deraction = " "
deraction = direction(all_sequences, num_col, line_index,third_line,1 )
print(deraction)
if deraction == None or deraction=="not found":
    print("if1")
    deraction = direction(all_sequences, num_col, line_index,third_line,2 )
    print(deraction)

    if deraction == None or deraction=="not found":
        print("if2")
        deraction = direction(all_sequences, num_col, line_index,third_line,3 )
print(deraction)



# #מדפיס ערכים של שורה ספציפית
# specific_line_sequences = all_sequences[0]
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
# # הדפסת ערכים של שורה ספציפית
# for sequence in specific_line_sequences:
#     row_index, start, length, value = sequence
#     matrix_position = (row_index, start)
#     print(f"Matrix Position: {matrix_position}, Length: {length}, Value: {value}")



cv2.imshow('Original Image', image)
cv2.imshow('Segmented Image', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
#חיבור של שניהם ביחד רק בסוף הסרטון מדפיס לאיפה הסימון מפנה.
import cv2
import numpy as np

#פונקציה לסיבוב תמונות
def rotateImage(sub_image):
    height, width = sub_image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), 45, 1)
    sub_image_top_right = cv2.warpAffine(sub_image, rotation_matrix, (width, height))

    rotation_matrix1 = cv2.getRotationMatrix2D((width / 2, height / 2), -45, 1)
    sub_image_top_left = cv2.warpAffine(sub_image, rotation_matrix1, (width, height))

   
    sub_image_horizontal = cv2.rotate(sub_image, cv2.ROTATE_90_CLOCKWISE)

    sub_image_vertical=sub_image
    
    return sub_image_top_right, sub_image_top_left,sub_image_horizontal,sub_image_vertical


def find_object_direction(video_path, sub_image_path):
    #הציון הטוב ביותר
    # best_match_score = float('inf')
    best_match_score = 0
    #המסגרת המתאימה ביותר 
    best_matching_frame = None

    sub_image = cv2.imread(sub_image_path)
#imread הופך תמונה למערך

    video = cv2.VideoCapture(video_path)
#VideoCapture ניתוח וידאו וחלוקה לפריימים
    best_matching_frame = video.read()
    
#קורא כל פעם פריים אחד מהוידאו לצורך חישוב
    ret, frame_prev = video.read()
#ret מחזיר אמת או שקר אם הפריים נקרא בהצלחה
#frame_prev מחזיר מערך המייצג את ערכי הפקסלים של הפריים שנקרא     
 #אתחול הערך למעלה שמאלה
    top_left_prev = None
    array_x = []
    array_y = []
    #מערך מונים לצורך כיוון התזוזה
    top_right1=0
    bottom_right1=0
    bottom_left1=0
    top_left1=0
    bottom1=0
    top1=0
    right1=0
    left1=0    
#אתחול כל משתני הכיוונים
    dir_x_total = 0
    dir_y_total = 0
    frame_count = 0    
#כל עוד שיש ברט ערך אמת שהמסגרת נקראה תבצע
    while ret:
        ret, frame_curr = video.read()
#אם אין עוד פריימים תצא
        if not ret:
            break
        
#זימון הפונקציה שמסובבת תמונות
        sub_image_top_right, sub_image_top_left,sub_image_horizontal,sub_image_vertical=rotateImage(sub_image)
        #מציאת התת תמונה לפי כל הכיוונים
        result_horizontal = cv2.matchTemplate(frame_curr, sub_image_horizontal, cv2.TM_CCOEFF_NORMED)     
        a1,max_horizontal,c1, max_loc_horizontal = cv2.minMaxLoc(result_horizontal)

        result_vertical = cv2.matchTemplate(frame_curr, sub_image_vertical, cv2.TM_CCOEFF_NORMED)     
        a2, max_vertical,c2, max_loc_vertical = cv2.minMaxLoc(result_vertical)

        result_top_right = cv2.matchTemplate(frame_curr, sub_image_top_right, cv2.TM_CCOEFF_NORMED)     
        a3, max_top_right,c3, max_loc_top_right = cv2.minMaxLoc(result_top_right)

        result_top_left = cv2.matchTemplate(frame_curr, sub_image_top_left, cv2.TM_CCOEFF_NORMED)     
        a4, max_top_left,c4, max_loc_top_left = cv2.minMaxLoc(result_top_left)

        #היכן מצא את ההתאמה הטובה ביותר
        # print(max_horizontal,max_vertical,max_top_right,max_top_left)
        
        max_value = max(max_horizontal,max_vertical,max_top_right,max_top_left)
        # print(max_value)
        if max_horizontal==max_value:
            top_left=max_loc_horizontal
        elif max_vertical==max_value:
            top_left=max_loc_vertical
        elif max_top_right==max_value:
            top_left=max_loc_top_right
        else:
            top_left=max_loc_top_left

        
 # top_left פינה למעלה שמאלה- מכילה את הערך המקסימלי      
      
#bottom_right פינה למטה ימינה
        bottom_right = (top_left[0] + sub_image.shape[1], top_left[1] + sub_image.shape[0])
#התמונה המינימלית
        cropped_image = frame_curr[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
#חיתוך התמונה
        cv2.imshow('Result', cropped_image)

        if  best_match_score < max_value:
            best_match_score = max_value
            best_matching_frame = cropped_image.copy()

        
        if top_left_prev is not None:
            direction_x = top_left[0] - top_left_prev[0]
            array_x.append(direction_x)
#חישוב הקצה העליון הנוכחי לעומת הקצה של הפריים הקודם
# כך מזהה את כיוון התנועה
            direction_y = top_left[1] - top_left_prev[1]
            array_y.append(direction_y)
            dir_x_total += direction_x
            dir_y_total += direction_y
            frame_count += 1
        
# מעדכן את הפריים הקודם- top_left_prev
        top_left_prev = top_left

#לצורך יציאה מלולאה כשלוחצים על התו הזה יוצא מהלולאה
        if cv2.waitKey(1) == ord('q'):
            break
#שחרור כל החלונות תצוגה שנפתחו
    video.release()
    cv2.destroyAllWindows()   
    for i in range(len(array_x)):
       # print( array_x[i] , array_y[i])

        if array_x[i]>0 and array_y[i]>0:
           bottom_right1=bottom_right1+1
        elif array_x[i] > 0 and array_y[i] < 0:
             top_right1= top_right1+1
        elif array_x[i] < 0 and array_y[i] > 0:
            bottom_left1=bottom_left1 +1
        elif array_x[i] < 0 and array_y[i] < 0:
             top_left1=top_left1+1
        elif array_x[i] > 0:
            right1=right1+1
        elif array_x[i] < 0:
            left1=left1+1
        elif array_y[i] > 0:
            bottom1=bottom1+1
        elif array_y[i] < 0:
            top1=top1+1
        else:
            direction = "stationary"
   #מציאת הערך המקסימלי והדפסתו
    directions = [bottom_right1,top_right1, bottom_left1, top_left1,right1,left1,bottom1, top1]
    directionsName = ['bottom_right1','top_right1', 'bottom_left1', 'top_left1','right1','left1','bottom1', 'top1']
    maxValue=-1000
    maxName=""
    for index, dir in enumerate(directions):
        if dir > maxValue:
            maxValue = dir
            maxName = directionsName[index]                
    print(f"Max direction: {maxName} with value {maxValue}") 
    return best_matching_frame

def segment_image(image, num_segments):
    # עצב מחדש את התמונה למערך דו-ממדי של פיקסלים
    pixels = image.reshape(-1, 3).astype(np.float32)

    # בצע אשכולות K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, num_segments, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # המר את המרכזים למספרים שלמים
    centers = np.uint8(centers)

    # עצב מחדש את התוויות כך שיתאימו לצורת התמונה המקורית
    segmented_image = centers[labels.flatten()].reshape(image.shape)

    # צור מטריצה ​​של סיווג קבוצתי עבור כל פיקסל
    group_matrix = labels.reshape(image.shape[:2])

    return segmented_image, group_matrix

#מוצאת רצפים
def sequences(group_matrix):
    all_sequences = []  # Initialize a list to store sequences for all rows
    
    # Iterate through each row in the matrix
    for row_index, row in enumerate(group_matrix):
        current = None
        start = None
        length = 0
        sequences = []

        # Repeat each value in the row
        for i, value in enumerate(row):
            if value != current:
                if current is not None:
                    sequences.append((row_index, start, length, current))
                current = value
                start = i
                length = 1
            else:
                length += 1
        
        sequences.append((row_index, start, length, current))
        
        all_sequences.append(sequences)  # Append sequences for this row to the list
    
    # Sort sequences by length and return only the three longest sequences
    all_sequences_sorted = []
    for row_sequences in all_sequences:
        row_sequences_sorted = sorted(row_sequences, key=lambda x: -x[2])[:3]  # Sort by length and get the 3 longest sequences
        # Sort the 3 longest sequences by start index within the row
        row_sequences_sorted = sorted(row_sequences_sorted, key=lambda x: x[1])
        all_sequences_sorted.append(row_sequences_sorted)
    
    return all_sequences_sorted



#מוצאת את הרצף הארוך ביותר
def long_sequence(specific_line_sequences):  
    #הסבר המשתנים-
    #longest_sequence - tuple שמכיל מידע על הרצף הארוך ביותר שנמצא בשורה שצוינה. מבנה הטפול הוא (row_index, start, length, value),
    #longest_sequence_length - מכיל את אורך הרצף הארוך ביותר שנמצא בשורה שצוינה
    longest_sequence = None
    longest_sequence_length = 0
    long = []
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        long.append(length)
        #הרצף הארוך ביותר
    longest_sequence_length = max(long)  
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        if length == longest_sequence_length:
            longest_sequence=sequence

    return longest_sequence


#מוצאת את הרצף במקום אינדקס
def sequence_in_index(specific_line_sequences, index): 
    sequences_count = 0
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        if length > 15:
            sequences_count += 1
            if sequences_count == index:
                return start, length, value
 
    return 100000, 0, 5


#פונקצית למציאת שיפוע- חיובי שלילי או שווה
def Incline (third_line, index_of_sequence):
    current_difference = 0
    prev_difference = 0
    positive = 0
    negative = 0
    equal = 0
    
    #רץ מהשליש השני של התמונה עד סוף השליש השני בקפיצות של 5
    for index in range(third_line, (third_line*2)):
         current_difference = (sequence_in_index (all_sequences[index], index_of_sequence))[0]
         result= current_difference - prev_difference
         if result > 0:
             positive += 1
         elif result < 0:
             negative += 1
         else:
             equal += 1
         prev_difference = current_difference 
    return positive, negative, equal

def compare(num1, num2):
    if abs(num1 - num2) <= 7:
        return True
    else:
        return False

#מוצאת כיוון      
def direction(all_sequences, num_col, line_index, third_line,seq_index) :
    print("from function")
    positive = 0
    negative = 0
    equal = 0
 #עבור רוחב
    col= 0.8 * num_col
    if (long_sequence(all_sequences[line_index])[2]>=col and long_sequence(all_sequences[line_index+20])[2]>=col or
        long_sequence(all_sequences[line_index+20])[2]>=col and long_sequence(all_sequences[line_index-20])[2]>=col):
        # אם הרצף הארוך ביותר של השורה האמצעית גדול מ80% מהאורך של השורה וגם הרצף ב50 שורות קודם ו50 שורות אחרי 
        return("רוחב")
 #עבור אלכסונים 
    positive, negative, equal = Incline (third_line, seq_index)
    print (equal)
    print (positive)
    print (negative)
    if equal >= positive and equal>= negative:
         positive, negative, equal = Incline (third_line, seq_index+1)
         print (equal)
         print (positive)
         print (negative)
         if equal >=positive and  equal>= negative:
             return ("אורך")
    
    elif  positive >= equal and positive>= negative:
        positive, negative, equal = Incline (third_line, seq_index)
        if  positive >= equal and positive>= negative:
            return ("אלכסון לשמאל")
    elif  negative >= equal and negative >= positive:
          positive, negative, equal = Incline (third_line, seq_index)
          if negative >= equal and negative >= positive:
              return ("אלכסון לימין")
    return "not found"



save = find_object_direction("./11.mp4","./tat.png")
# cv2.imshow('Best Matching Frame', save)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# image = cv2.imread('./left.jpg')
image = save

num_segments =3
segmented_image, group_matrix1 = segment_image(image, num_segments)
#segmented_image-מטריצה שמייצגת את התמונה המסווגת
#group_matrix1- מטריצה שמייצגת עבור כל פיקסל בתמונה לאיזו קבוצה הוא משתייך

np.set_printoptions(threshold=np.inf)

 #חישוב כמה העמודות
num_col = len(group_matrix1[0])

#זימון פונקצית הרצפים
all_sequences = sequences(group_matrix1)

line_index = round(len(group_matrix1)/2)
#חישוב מיקום השליש השני במטריצה
third_line = len(group_matrix1)//3
deraction = " "
deraction = direction(all_sequences, num_col, line_index,third_line,1 )
print(deraction)
if deraction == None or deraction=="not found":
    print("if1")
    deraction = direction(all_sequences, num_col, line_index,third_line,2 )
    print(deraction)

    if deraction == None or deraction=="not found":
        print("if2")
        deraction = direction(all_sequences, num_col, line_index,third_line,3 )
print(deraction)


cv2.imshow('Original Image', image)
cv2.imshow('Segmented Image', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Max direction: top_right1 with value 30
from function
13
6
1
6
13
1
not found
if1
from function
6
13
1
אלכסון לשמאל
אלכסון לשמאל


In [ ]:
#?????????????????????????????????????????????????????????????????????

import cv2
import numpy as np

#פונקציה לסיבוב תמונות
def rotateImage(sub_image):
    height, width = sub_image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), 45, 1)
    sub_image_top_right = cv2.warpAffine(sub_image, rotation_matrix, (width, height))

    rotation_matrix1 = cv2.getRotationMatrix2D((width / 2, height / 2), -45, 1)
    sub_image_top_left = cv2.warpAffine(sub_image, rotation_matrix1, (width, height))

   
    sub_image_horizontal = cv2.rotate(sub_image, cv2.ROTATE_90_CLOCKWISE)

    sub_image_vertical=sub_image
    
    return sub_image_top_right, sub_image_top_left,sub_image_horizontal,sub_image_vertical


def find_object_direction(video_path, sub_image_path):
    #הציון הטוב ביותר
    # best_match_score = float('inf')
    best_match_score = 0
    #המסגרת המתאימה ביותר 
    best_matching_frame = None

    sub_image = cv2.imread(sub_image_path)
#imread הופך תמונה למערך

    video = cv2.VideoCapture(video_path)
#VideoCapture ניתוח וידאו וחלוקה לפריימים
    best_matching_frame = video.read()
    
#קורא כל פעם פריים אחד מהוידאו לצורך חישוב
    ret, frame_prev = video.read()
#ret מחזיר אמת או שקר אם הפריים נקרא בהצלחה
#frame_prev מחזיר מערך המייצג את ערכי הפקסלים של הפריים שנקרא     
 #אתחול הערך למעלה שמאלה
    top_left_prev = None
    array_x = []
    array_y = []
    #מערך מונים לצורך כיוון התזוזה
    top_right1=0
    bottom_right1=0
    bottom_left1=0
    top_left1=0
    bottom1=0
    top1=0
    right1=0
    left1=0    
#אתחול כל משתני הכיוונים
    dir_x_total = 0
    dir_y_total = 0
    frame_count = 0    
#כל עוד שיש ברט ערך אמת שהמסגרת נקראה תבצע
    while ret:
        ret, frame_curr = video.read()
#אם אין עוד פריימים תצא
        if not ret:
            break
        
#זימון הפונקציה שמסובבת תמונות
        sub_image_top_right, sub_image_top_left,sub_image_horizontal,sub_image_vertical=rotateImage(sub_image)
        #מציאת התת תמונה לפי כל הכיוונים
        result_horizontal = cv2.matchTemplate(frame_curr, sub_image_horizontal, cv2.TM_CCOEFF_NORMED)     
        a1,max_horizontal,c1, max_loc_horizontal = cv2.minMaxLoc(result_horizontal)

        result_vertical = cv2.matchTemplate(frame_curr, sub_image_vertical, cv2.TM_CCOEFF_NORMED)     
        a2, max_vertical,c2, max_loc_vertical = cv2.minMaxLoc(result_vertical)

        result_top_right = cv2.matchTemplate(frame_curr, sub_image_top_right, cv2.TM_CCOEFF_NORMED)     
        a3, max_top_right,c3, max_loc_top_right = cv2.minMaxLoc(result_top_right)

        result_top_left = cv2.matchTemplate(frame_curr, sub_image_top_left, cv2.TM_CCOEFF_NORMED)     
        a4, max_top_left,c4, max_loc_top_left = cv2.minMaxLoc(result_top_left)

        #היכן מצא את ההתאמה הטובה ביותר
        print(max_horizontal,max_vertical,max_top_right,max_top_left)
        
        max_value = max(max_horizontal,max_vertical,max_top_right,max_top_left)
        print(max_value)
        if max_horizontal==max_value:
            top_left=max_loc_horizontal
        elif max_vertical==max_value:
            top_left=max_loc_vertical
        elif max_top_right==max_value:
            top_left=max_loc_top_right
        else:
            top_left=max_loc_top_left

        
 # top_left פינה למעלה שמאלה- מכילה את הערך המקסימלי      
      
#bottom_right פינה למטה ימינה
        bottom_right = (top_left[0] + sub_image.shape[1], top_left[1] + sub_image.shape[0])
#התמונה המינימלית
        cropped_image = frame_curr[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
#חיתוך התמונה
        cv2.imshow('Result', cropped_image)

        if  best_match_score < max_value:
            best_match_score = max_value
            best_matching_frame = cropped_image.copy()

        
        if top_left_prev is not None:
            direction_x = top_left[0] - top_left_prev[0]
            array_x.append(direction_x)
#חישוב הקצה העליון הנוכחי לעומת הקצה של הפריים הקודם
# כך מזהה את כיוון התנועה
            direction_y = top_left[1] - top_left_prev[1]
            array_y.append(direction_y)
            dir_x_total += direction_x
            dir_y_total += direction_y
            frame_count += 1
        
# מעדכן את הפריים הקודם- top_left_prev
        top_left_prev = top_left

#לצורך יציאה מלולאה כשלוחצים על התו הזה יוצא מהלולאה
        if cv2.waitKey(1) == ord('q'):
            break
#שחרור כל החלונות תצוגה שנפתחו
    video.release()
    cv2.destroyAllWindows()   
    for i in range(len(array_x)):
       # print( array_x[i] , array_y[i])

        if array_x[i]>0 and array_y[i]>0:
           bottom_right1=bottom_right1+1
        elif array_x[i] > 0 and array_y[i] < 0:
             top_right1= top_right1+1
        elif array_x[i] < 0 and array_y[i] > 0:
            bottom_left1=bottom_left1 +1
        elif array_x[i] < 0 and array_y[i] < 0:
             top_left1=top_left1+1
        elif array_x[i] > 0:
            right1=right1+1
        elif array_x[i] < 0:
            left1=left1+1
        elif array_y[i] > 0:
            bottom1=bottom1+1
        elif array_y[i] < 0:
            top1=top1+1
        else:
            direction = "stationary"
   #מציאת הערך המקסימלי והדפסתו
    directions = [bottom_right1,top_right1, bottom_left1, top_left1,right1,left1,bottom1, top1]
    directionsName = ['bottom_right1','top_right1', 'bottom_left1', 'top_left1','right1','left1','bottom1', 'top1']
    maxValue=-1000
    maxName=""
    for index, dir in enumerate(directions):
        if dir > maxValue:
            maxValue = dir
            maxName = directionsName[index]                
    print(f"Max direction: {maxName} with value {maxValue}") 
    return best_matching_frame

def segment_image(image, num_segments):
    # עצב מחדש את התמונה למערך דו-ממדי של פיקסלים
    pixels = image.reshape(-1, 3).astype(np.float32)

    # בצע אשכולות K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(pixels, num_segments, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # המר את המרכזים למספרים שלמים
    centers = np.uint8(centers)

    # עצב מחדש את התוויות כך שיתאימו לצורת התמונה המקורית
    segmented_image = centers[labels.flatten()].reshape(image.shape)

    # צור מטריצה ​​של סיווג קבוצתי עבור כל פיקסל
    group_matrix = labels.reshape(image.shape[:2])

    return segmented_image, group_matrix

#מוצאת רצפים
def sequences(group_matrix):
    all_sequences = []  # Initialize a list to store sequences for all rows
    
    # Iterate through each row in the matrix
    for row_index, row in enumerate(group_matrix):
        current = None
        start = None
        length = 0
        sequences = []

        # Repeat each value in the row
        for i, value in enumerate(row):
            if value != current:
                if current is not None:
                    sequences.append((row_index, start, length, current))
                current = value
                start = i
                length = 1
            else:
                length += 1
        
        sequences.append((row_index, start, length, current))
        
        all_sequences.append(sequences)  # Append sequences for this row to the list
    
    # Sort sequences by length and return only the three longest sequences
    all_sequences_sorted = []
    for row_sequences in all_sequences:
        row_sequences_sorted = sorted(row_sequences, key=lambda x: -x[2])[:3]  # Sort by length and get the 3 longest sequences
        # Sort the 3 longest sequences by start index within the row
        row_sequences_sorted = sorted(row_sequences_sorted, key=lambda x: x[1])
        all_sequences_sorted.append(row_sequences_sorted)

    
    return all_sequences_sorted


#מוצאת את הרצף הארוך ביותר
def long_sequence(specific_line_sequences):  
    #הסבר המשתנים-
    #longest_sequence - tuple שמכיל מידע על הרצף הארוך ביותר שנמצא בשורה שצוינה. מבנה הטפול הוא (row_index, start, length, value),
    #longest_sequence_length - מכיל את אורך הרצף הארוך ביותר שנמצא בשורה שצוינה
    longest_sequence = None
    longest_sequence_length = 0
    long = []
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        long.append(length)
        #הרצף הארוך ביותר
    longest_sequence_length = max(long)  
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        if length == longest_sequence_length:
            longest_sequence=sequence

    return longest_sequence


#מוצאת את הרצף במקום אינדקס
def sequence_in_index(specific_line_sequences, index): 
    sequences_count = 0
    for sequence in specific_line_sequences:
        row_index, start, length, value = sequence
        if length > 15:
            sequences_count += 1
            if sequences_count == index:
                return start, length, value
 
    return 100000, 0, 5


#פונקצית למציאת שיפוע- חיובי שלילי או שווה
def Incline (third_line, index_of_sequence):
    current_difference = 0
    prev_difference = 0
    positive = 0
    negative = 0
    equal = 0
    
    #רץ מהשליש השני של התמונה עד סוף השליש השני בקפיצות של 5
    for index in range(third_line, (third_line*2), 5):
         current_difference = (sequence_in_index (all_sequences[index], index_of_sequence))[0]
         result= current_difference - prev_difference
         if result > 0:
             positive += 1
         elif result < 0:
             negative += 1
         else:
             equal += 1
         prev_difference = current_difference 
    return positive, negative, equal 

def compare(num1, num2):
    if abs(num1 - num2) <= 7:
        return True
    else:
        return False

#מוצאת כיוון      
def direction(all_sequences, num_col, line_index, third_line) :
    print("from function")
    positive = 0
    negative = 0
    equal = 0
 #עבור רוחב
    col= 0.8 * num_col
    if (long_sequence(all_sequences[line_index])[2]>=col and long_sequence(all_sequences[line_index+20])[2]>=col or
        long_sequence(all_sequences[line_index+20])[2]>=col and long_sequence(all_sequences[line_index-20])[2]>=col):
        # אם הרצף הארוך ביותר של השורה האמצעית גדול מ80% מהאורך של השורה וגם הרצף ב50 שורות קודם ו50 שורות אחרי 
        return("רוחב")
 #עבור אלכסונים 
    positive, negative, equal = Incline (third_line, 1)
    if equal >= (positive + negative):
         positive, negative, equal = Incline (third_line, 2)
         if equal >= (positive + negative):
             return ("אורך")
         else:
              positive, negative, equal = Incline (third_line, 2)
              if equal >= positive> negative:
                 return ("אורך")
    elif  positive >= (equal + negative):
        positive, negative, equal = Incline (third_line, 3)
        if  positive >= (equal + negative):
            return ("אלכסון לשמאל")
        else:
            positive, negative, equal = Incline (third_line, 4)
            if positive > equal > negative:
                 return ("אלכסון לשמאל")
    elif  negative >= (equal + positive):
          positive, negative, equal = Incline (third_line, 3)
          if negative >= (equal + positive):
              return ("אלכסון לימין")
          else:
              positive, negative, equal = Incline (third_line, 4)
              if negative > equal > positive:
                 return ("אלכסון לימין")
    else:
        return "not found"



save = find_object_direction("./42.mp4","./tat.png")
# cv2.imshow('Best Matching Frame', save)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# image = cv2.imread('./left.jpg')
image = save

num_segments =3
segmented_image, group_matrix1 = segment_image(image, num_segments)
#segmented_image-מטריצה שמייצגת את התמונה המסווגת
#group_matrix1- מטריצה שמייצגת עבור כל פיקסל בתמונה לאיזו קבוצה הוא משתייך

np.set_printoptions(threshold=np.inf)

 #חישוב כמה העמודות
num_col = len(group_matrix1[0])

#זימון פונקצית הרצפים
all_sequences = sequences(group_matrix1)

line_index = round(len(group_matrix1)/2)
#חישוב מיקום השליש השני במטריצה
third_line = len(group_matrix1)//3
deraction = " "
deraction = direction(all_sequences, num_col, line_index,third_line )
print(deraction)

cv2.imshow('Original Image', image)
cv2.imshow('Segmented Image', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
from playsound import playsound
 
# for playing note.wav file
playsound('./111.mp3')
print('playing sound using  playsound')

In [ ]:
!pip install --trusted-host files.pythonhosted.org --trusted-host pypi.org --trusted-host pypi.python.org oauthlib -vvv playsound

In [3]:
!pip show pygame

Name: pygame
Version: 2.5.2
Summary: Python Game Development
Home-page: https://www.pygame.org
Author: A community project.
Author-email: pygame@pygame.org
License: LGPL
Location: c:\users\comp2\appdata\local\programs\python\python310\lib\site-packages
Requires: 
Required-by: 


In [ ]:
!pip install --trusted-host files.pythonhosted.org --trusted-host pypi.org --trusted-host pypi.python.org playsound


In [ ]:
import os

file = "111.mp3"
os.system("X:\הנדסאים שנה ב תשפד\פרויקטים\הודיה שרגא\חיבור הקודים\111.mp3")


In [ ]:
from pydub import AudioSegment
from pydub.playback import play

sound = AudioSegment.from_file(r'f:\aaa.wav','wav')
play(sound)

In [ ]:
!pip install --trusted-host files.pythonhosted.org --trusted-host pypi.org --trusted-host pypi.python.org pygame


In [4]:
import pygame
pygame.init()
my_sound = pygame.mixer.Sound('f:\\aaa.wav')
my_sound.play()

ModuleNotFoundError: No module named 'pygame'

In [6]:
import sys

# Print out the contents of sys.path
print("Contents of sys.path:")
for path in sys.path:
    print(path)

# Attempt to import pygame
try:
    import pygame
    print("pygame imported successfully")
except ImportError:
    print("Unable to import pygame")


Contents of sys.path:
X:\הנדסאים שנה ב תשפד\פרויקטים\הודיה שרגא\חיבור הקודים
C:\Users\comp2\AppData\Local\Programs\Python\Python312\python312.zip
C:\Users\comp2\AppData\Local\Programs\Python\Python312\DLLs
C:\Users\comp2\AppData\Local\Programs\Python\Python312\Lib
C:\Users\comp2\AppData\Local\Programs\Python\Python312

C:\Users\comp2\AppData\Local\Programs\Python\Python312\Lib\site-packages
C:\Users\comp2\AppData\Local\Programs\Python\Python312\Lib\site-packages\win32
C:\Users\comp2\AppData\Local\Programs\Python\Python312\Lib\site-packages\win32\lib
C:\Users\comp2\AppData\Local\Programs\Python\Python312\Lib\site-packages\Pythonwin
Unable to import pygame
